In [1]:
# Import necessary linbraries
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import os
import smtplib
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders

# Initialize Tkinter
root = tkinter.Tk()
root.withdraw()

#Load trained deep learning model
model = load_model('face_mask_detection_alert_system.h5')

#Classifier to detect face
face_det_classifier=cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

# Capture Video
vid_source=cv2.VideoCapture(0)

# Dictionaries containing details of Wearing Mask and Color of rectangle around face. If wearing mask then color would be 
# green and if not wearing mask then color of rectangle around face would be red
text_dict={0:'Mask ON',1:'No Mask'}
rect_color_dict={0:(0,255,0),1:(255,0,0)}

SUBJECT = "Subject"   
TEXT = "One Visitor violated Face Mask Policy. See in the camera to recognize user. A Person has been detected without a face mask in the Hotel Lobby Area 9. Please Alert the authorities."
 

# While Loop to continuously detect camera feed
count = 0
while(True):

    ret, img = vid_source.read()
    grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img,1.3,5)
    cnt=1

    for (x,y,w,h) in faces:
    
        face_img = grayscale_img[y:y+w,x:x+w]
        resized_img = cv2.resize(face_img,(112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img,(1,112,112,1))
        result=model.predict(reshaped_img)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img, text_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2) 
        
        
        # If label = 1 then it means wearing No Mask and 0 means wearing Mask
        if (label == 1):
            path=r'..\Face Mask Detection and alerting system\Offender\Offender' +str(cnt)+'.jpg'
            cv2.imwrite(path,img)
            cnt += 1
            # Throw a Warning Message to tell user to wear a mask if not wearing one. This will stay
            #open and No Access will be given He/She wears the mask
            messagebox.showwarning("Warning","Access Denied. Please wear a Face Mask")
            
            # Send an email to the administrator if access denied/user not wearing face mask 
            fromaddr = "sender's_email_id"
            toaddr = "receiver's_email_id"
            
            # instance of MIMEMultipart
            msg = MIMEMultipart()
            
            # Storing the sender's email address
            
            msg['From'] = fromaddr 
  
            # storing the receivers email address  
            msg['To'] = toaddr 

            # storing the subject  
            msg['Subject'] = "Offense has been Captured "

            # string to store the body of the mail 
            body = "One Visitor violated Face Mask Policy. See in the camera to recognize user. A Person has been detected without a face mask in the Hotel Lobby Area 9. Please Alert the authorities."

            # attach the body with the msg instance 
            msg.attach(MIMEText(body, 'plain')) 

            # open the file to be sent  
            filename = "Offender1.jpg"
            attachment = open(r"..\Face Mask Detection and alerting system\Offender\Offender1.jpg", "rb") 

            # instance of MIMEBase and named as p 
            p = MIMEBase('application', 'octet-stream') 

            # To change the payload into encoded form 
            p.set_payload((attachment).read())

            # encode into base64 
            encoders.encode_base64(p) 

            p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 

            # attach the instance 'p' to instance 'msg' 
            msg.attach(p) 

            # creates SMTP session 
            s = smtplib.SMTP('smtp.gmail.com', 587) 

            # start TLS for security 
            s.starttls() 

            # Authentication 
            s.login(fromaddr, "your_password") 

            # Converts the Multipart msg into a string 
            text = msg.as_string() 

            # sending the mail 
            s.sendmail(fromaddr, toaddr, text) 

            # terminating the session 
            s.quit() 
        else:
            pass
            break

    cv2.imshow('Camera 1',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
vid_source.release()
